In [1]:
import requests
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import re
import os
import json
import time

In [2]:
from multiprocessing import Pool

In [3]:
def download_instagram_json(shortcode):
    out_path = "/mnt/e/Data/selfies/instagram_json"
    out_file = shortcode + ".json"
    if not os.path.exists(os.path.join(out_path, out_file)):
        try:
            print("Processing:", shortcode)
            r  = requests.get("https://www.instagram.com/p/" + shortcode +"?__a=1")
            data = r.json()

            # Get Likes, Following, Followers and append to data
            owner_username = data["graphql"]["shortcode_media"]["owner"]["username"]
            owner_url = "https://www.instagram.com/" + owner_username
            r = requests.get(owner_url)
            soup = BeautifulSoup(r.text, "lxml")
            description = soup.find("meta", property="og:description")["content"]
            nums = re.compile(r"[+-]?\d+(?:\.\d+)?")
            res = nums.findall(description.split("-")[0])
            owner_stats = dict(zip(["followers", "following", "posts"], res))
            data["graphql"]["shortcode_media"]["owner"].update(owner_stats)

            # Save data
            with open(os.path.join(out_path, out_file), 'w') as fout:
                json.dump(data, fout)
        except Exception as e:
            print("Error:", shortcode, e)

In [4]:
def main():
    in_path = "/mnt/e/Data/selfies/images"
    out_path = "/mnt/e/Data/selfies/face_api_json"

    shortcodes_analysed_json = os.listdir(in_path)
    shortcodes_analysed = list(map(lambda x: x[:-4], shortcodes_analysed_json))
    
    for shortcode in shortcodes_analysed:
        download_instagram_json(shortcode)
#     # Multiprocess images
#     with Pool(6) as p:
#         p.map(download_instagram_json, shortcodes)

In [ ]:
if __name__ == '__main__':
    main()